In [ ]:
%pip install --upgrade google-cloud-aiplatform tensorboard protobuf python-magic

In [ ]:
import base64
import io
import magic
from matplotlib.pyplot import flag
import numpy as np
from sympy import false
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Part
import gradio as gr
from PIL import Image

vertexai.init(project="chat-380108")
model = GenerativeModel("gemini-pro-vision")
def compress_image(file_path, output_format='JPEG', quality=85):
    # Open the original image
    with Image.open(file_path) as img:
        # Optionally, resize the image here if needed
        img = img.resize((512, 512))

        # Save the image to a bytes buffer with specified format and quality
        img_byte_arr = io.BytesIO()
        img.save(img_byte_arr, format=output_format, quality=quality)
        img_byte_arr = img_byte_arr.getvalue()

    return img_byte_arr

def generate(file_path):
    history = ""
    with open(file_path, "rb") as file:
        data = file.read()

    # Use magic to get the MIME type from the file path
    mime = magic.Magic(mime=True)
    mime_type = mime.from_file(file_path)

    responses = model.generate_content(
        [
            Part.from_data(data=data, mime_type=mime_type),
            """Extract this in json.
        if its the photo of a shipping label for a  vehicle here the format
        {
        "Prod. Month": "202304",
        "Destination Code": "45722",
        "Port Date Arrival": "SOUTH AFRICA",
        "Country Name": "1",
        "Engine No.": "3XTRH102364",
        "Body No.": "Assembly Date",
        "ED No.": "7GHJL34",
        "Katashaki/Model": "G900XR-TNKM4",
        "Frame Code": "B12 BLACK B12",
        "DS No.": "782",
        "Body Color Code and Name": "YHL9B3X45",
        "VIN No.": "XP334577",
        "VLC Yard": "Distribution center",
        "Marshalling Yard": "75HJK",
        "Assign Code": "X2-09"
        }
        else return the json you want describing the image
        Return only the json""",
        ],
        generation_config={
            "max_output_tokens": 2048,
            "temperature": 0.3,
            "top_p": 0.8,
            "top_k": 32,
        },
        stream=True,
    )

    for response in responses:
        history += response.candidates[0].content.parts[0].text
        yield history


iface = gr.Interface(
    fn=generate,
    inputs=gr.Image(sources=["upload", "webcam"], type="filepath"),
    outputs="text",
    allow_flagging='never'
)

if __name__ == "__main__":
    iface.queue()
    iface.launch()